In [3]:
import os

for cls in os.listdir(DATA_DIR):
    cls_path =_


In [4]:
# =======================
# Dataset check
# =======================

print("Checking dataset folder structure and image counts...")

total_images = 0
empty_classes = []

for cls in os.listdir(DATA_DIR):
    cls_path = os.path.join(DATA_DIR, cls)
    if os.path.isdir(cls_path):
        num_imgs = len([f for f in os.listdir(cls_path) 
                        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff', '.gif'))])
        print(f"Class '{cls}': {num_imgs} images")
        total_images += num_imgs
        if num_imgs == 0:
            empty_classes.append(cls)

if total_images == 0:
    raise ValueError(f"No images found in '{DATA_DIR}'. Check your dataset path and folder structure!")

if empty_classes:
    print(f"Warning: The following classes are empty: {empty_classes}")
    print("These classes will be ignored by the generator.")


Checking dataset folder structure and image counts...
Class 'A': 3 images
Class 'B': 3 images
Class 'C': 3 images
Class 'D': 3 images
Class 'E': 3 images
Class 'F': 3 images
Class 'G': 3 images
Class 'H': 3 images
Class 'I': 3 images
Class 'J': 3 images
Class 'K': 3 images
Class 'L': 3 images
Class 'M': 3 images
Class 'N': 3 images
Class 'O': 3 images
Class 'P': 3 images
Class 'Q': 3 images
Class 'R': 3 images
Class 'S': 3 images
Class 'T': 3 images
Class 'U': 3 images
Class 'V': 3 images
Class 'W': 3 images
Class 'X': 3 images
Class 'Y': 3 images
Class 'Z': 3 images


In [11]:
import os
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
import sklearn.metrics as skm

# =======================
# Config
# =======================

DATA_DIR = "labelled"          
MODEL_DIR = "models"
LOG_DIR = os.path.join(MODEL_DIR, "logs")
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

IMG_SIZE = (128, 128)        
BATCH_SIZE = 16
EPOCHS = 20                  
SEED = 42
LEARNING_RATE = 1e-4
NUM_CLASSES = 26             

# =======================
# Dataset check
# =======================

print("Checking dataset folder structure and image counts...")

total_images = 0
empty_classes = []

for cls in os.listdir(DATA_DIR):
    cls_path = os.path.join(DATA_DIR, cls)
    if os.path.isdir(cls_path):
        num_imgs = len([f for f in os.listdir(cls_path) 
                        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff', '.gif'))])
        print(f"Class '{cls}': {num_imgs} images")
        total_images += num_imgs
        if num_imgs == 0:
            empty_classes.append(cls)

if total_images == 0:
    raise ValueError(f"No images found in '{DATA_DIR}'. Check your dataset path and folder structure!")

if empty_classes:
    print(f"Warning: The following classes are empty and will be ignored: {empty_classes}")

# =======================
# Data generators with augmentation
# =======================

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=15,
    width_shift_range=0.10,
    height_shift_range=0.10,
    shear_range=0.05,
    zoom_range=0.10,
    horizontal_flip=True,
    brightness_range=(0.8, 1.2),
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
    seed=SEED
)

# Use same dataset as "validation" (or optionally hold out a few images manually)
validation_generator = val_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False,
    seed=SEED
)

# =======================
# Compute class weights
# =======================

y_train_labels = train_generator.classes  
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train_labels),
    y=y_train_labels
)
class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)

# =======================
# Model definition
# =======================

def build_compact_cnn(input_shape=IMG_SIZE + (3,), num_classes=NUM_CLASSES):
    inp = layers.Input(shape=input_shape)

    # Block 1
    x = layers.Conv2D(32, (3,3), padding="same", activation="relu")(inp)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2,2))(x)

    # Block 2
    x = layers.Conv2D(64, (3,3), padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2,2))(x)

    # Block 3
    x = layers.Conv2D(128, (3,3), padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2,2))(x)

    # Global pooling
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=inp, outputs=outputs, name="compact_sasl_cnn")
    return model

model = build_compact_cnn()
model.summary()

# =======================
# Compile model
# =======================

optimizer = optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# =======================
# Callbacks
# =======================

checkpoint_path = os.path.join(MODEL_DIR, "sasl_cnn.keras")
cb_checkpoint = callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)
cb_earlystop = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=8,
    restore_best_weights=True,
    verbose=1
)
cb_reduce_lr = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=4,
    min_lr=1e-7,
    verbose=1
)
tensorboard_cb = callbacks.TensorBoard(log_dir=LOG_DIR)

# =======================
# Train
# =======================

steps_per_epoch = math.ceil(train_generator.samples / BATCH_SIZE)
validation_steps = math.ceil(validation_generator.samples / BATCH_SIZE)

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    class_weight=class_weights,
    callbacks=[cb_checkpoint, cb_earlystop, cb_reduce_lr, tensorboard_cb],
    verbose=1
)

# =======================
# Save final model
# =======================

model.save(os.path.join(MODEL_DIR, "finalX1_sasl_cnn.keras"))


Checking dataset folder structure and image counts...
Class 'A': 3 images
Class 'B': 3 images
Class 'C': 3 images
Class 'D': 3 images
Class 'E': 3 images
Class 'F': 3 images
Class 'G': 3 images
Class 'H': 3 images
Class 'I': 3 images
Class 'J': 3 images
Class 'K': 3 images
Class 'L': 3 images
Class 'M': 3 images
Class 'N': 3 images
Class 'O': 3 images
Class 'P': 3 images
Class 'Q': 3 images
Class 'R': 3 images
Class 'S': 3 images
Class 'T': 3 images
Class 'U': 3 images
Class 'V': 3 images
Class 'W': 3 images
Class 'X': 3 images
Class 'Y': 3 images
Class 'Z': 3 images
Found 78 images belonging to 26 classes.
Found 78 images belonging to 26 classes.
Class weights: {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0, 8: 1.0, 9: 1.0, 10: 1.0, 11: 1.0, 12: 1.0, 13: 1.0, 14: 1.0, 15: 1.0, 16: 1.0, 17: 1.0, 18: 1.0, 19: 1.0, 20: 1.0, 21: 1.0, 22: 1.0, 23: 1.0, 24: 1.0, 25: 1.0}


Model: "compact_sasl_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_21 (Conv2D)                   │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_21               │ (None, 128, 128, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_21 (MaxPooling2D)      │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_22 (Conv2D)                   │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_22               │ (None, 64, 64, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_22 (MaxPooling2D)      │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_23 (Conv2D)                   │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_23               │ (None, 32, 32, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_23 (MaxPooling2D)      │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_7           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 26)                  │           3,354 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 114,010 (445.35 KB)

 Trainable params: 113,562 (443.60 KB)

 Non-trainable params: 448 (1.75 KB)

C:\Users\Seithati\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step - accuracy: 0.0000e+00 - loss: 3.5185
Epoch 1: val_accuracy improved from -inf to 0.03846, saving model to models\sasl_cnn.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 726ms/step - accuracy: 0.0000e+00 - loss: 3.5199 - val_accuracy: 0.0385 - val_loss: 3.2714 - learning_rate: 1.0000e-04
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step - accuracy: 0.0161 - loss: 3.3008
Epoch 2: val_accuracy improved from 0.03846 to 0.05128, saving model to models\sasl_cnn.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 629ms/step - accuracy: 0.0156 - loss: 3.3133 - val_accuracy: 0.0513 - val_loss: 3.2713 - learning_rate: 1.0000e-04
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 457ms/step - accuracy: 0.0949 - loss: 3.4539
Epoch 3: val_accuracy did not improve from 0.05128
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 624ms/step - accuracy: 0.0898 - loss: 3.4444 - val_accuracy: 0.0385 - val_loss: 3.2712 - learning_rate: 1.0000e-04
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 418ms/step - accuracy: 0.0116 - loss

In [12]:

# =======================
# Evaluate on validation set
# =======================

val_loss, val_acc = model.evaluate(validation_generator, steps=validation_steps, verbose=1)
print(f"Validation accuracy: {val_acc:.4f}, Validation loss: {val_loss:.4f}")

# =======================
# Classification report & confusion matrix
# =======================

y_true = []
y_pred = []
validation_generator.reset()
for i in range(validation_steps):
    x_batch, y_batch = next(validation_generator)
    preds = model.predict(x_batch)
    y_true.extend(np.argmax(y_batch, axis=1).tolist())
    y_pred.extend(np.argmax(preds, axis=1).tolist())

idx_to_class = {v:k for k,v in train_generator.class_indices.items()}

cm = skm.confusion_matrix(y_true, y_pred)
report = skm.classification_report(y_true, y_pred, target_names=[idx_to_class[i] for i in range(len(idx_to_class))], zero_division=0)
print("Classification report:\n", report)
print("Confusion matrix:\n", cm)

np.save(os.path.join(MODEL_DIR, "confusion_matrix.npy"), cm)


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 148ms/step - accuracy: 0.0310 - loss: 3.2742
Validation accuracy: 0.0385, Validation loss: 3.2711
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
Classification report:
               precision    recall  f1-score   support

           A       0.00      0.00      0.00         3
           B       0.00      0.00      0.00         3
           C       0.00      0.00      0.00         3
           D       0.00      0.00      0.00         3
           E       0.00      0.00      0.00         3
           F       0.00      0.00      0.00         3
           G       0.00      0.00      0.00         3
           H       0.00      0.00      0.00         3
           I       0.00      0.00      0.00         3
           J       0.00      0.00      0.00         3
           K       0.00      0.00      0.00         3
        